<a href="https://colab.research.google.com/github/duyvm/funny_stuff_with_llm/blob/main/learning-rag/Langchain_multiqueryretriever.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install --quiet --upgrade langchain-chroma langchain[openai] langchain langchain-community langgraph langchain-core langchain-text-splitters> /dev/null

In [ ]:
from google.colab import userdata
import os

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGSMITH_PROJECT"] = f"langchain-learning-rag"
os.environ["LANGSMITH_API_KEY"] = userdata.get('LANGSMITH_API_KEY')
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

# Overview

Guide: [How to use MultiQueryRetriever](https://python.langchain.com/docs/how_to/MultiQueryRetriever/)

- Retrieval may produces different results if there is subtle changes in query wordings or embeddings do not capture semantic meaning of data.

- Using `MultiQueryRetriever` automates the process of prompt tuning by leveraging llm to generate multiple queries from different perspectives for a user input.

**Detail methods**

1. For given query, use llm to generate multiple queries

2. Retrieve documents using those queries

3. Union unique documents from retrieval results and pass them as context for answer synthesis

# Load data

Using the [LLM Powered Autonomous Agents](https://lilianweng.github.io/posts/2023-06-23-agent/) blog post by Lilian Weng

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_openai import OpenAIEmbeddings

loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
data = loader.load()

splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
docs = splitter.split_documents(data)

embedding = OpenAIEmbeddings()
vectordb = Chroma.from_documents(docs, embedding)

In [ ]:
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_openai import ChatOpenAI

question = "What are the approaches to Task Decomposition?"
llm = ChatOpenAI(temperature=0)
retriever_from_llm = MultiQueryRetriever.from_llm(
    retriever=vectordb.as_retriever(), llm=llm
)

In [ ]:
import logging

logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

In [ ]:
unique_docs = retriever_from_llm.invoke(question)
len(unique_docs)

INFO:langchain.retrievers.multi_query:Generated queries: ['1. How can Task Decomposition be achieved through different methods?', '2. What strategies can be used for breaking down tasks in Task Decomposition?', '3. What are the various techniques for approaching Task Decomposition effectively?']


6

- Default `MultiQueryRetriver` prompt: [MultiQueryRetriever](https://python.langchain.com/api_reference/langchain/retrievers/langchain.retrievers.multi_query.MultiQueryRetriever.html)

- Use your own prompt:

  1. Make a `PromptTemplate`

  2. Implement an output parser

In [ ]:
from typing import List
from langchain_core.output_parsers import BaseOutputParser
from langchain_core.prompts import PromptTemplate
from pydantic import BaseModel, Field

class LineListOutputParse(BaseOutputParser[List[str]]):
    """Output parser for list of lines."""
    def parse(self, text: str) -> List[str]:
        """Parse the output of an LLM call."""
        lines = text.strip().split("\n")
        return list(filter(None, lines))

ouput_parser = LineListOutputParse()

QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are an AI language model assistant. Your task is to generate five
    different versions of the given user question to retrieve relevant documents from a vector
    database. By generating multiple perspectives on the user question, your goal is to help
    the user overcome some of the limitations of the distance-based similarity search.
    Provide these alternative questions seperated by newlines.
    Original question: {question}
    """,
)

llm = ChatOpenAI(temperature=0)

chain = QUERY_PROMPT | llm | ouput_parser

In [ ]:
retriever = MultiQueryRetriever(
    retriever=vectordb.as_retriever(),
    llm_chain=chain,
    parser_key="lines",
)

unique_docs = retriever.invoke("What does the course say about regression?")
len(unique_docs)

INFO:langchain.retrievers.multi_query:Generated queries: ['1. Can you provide information on regression as discussed in the course?', '2. What topics related to regression are covered in the course material?', '3. How is regression explained in the course content?', '4. What insights does the course offer on regression?', '5. What are the key points regarding regression in the course curriculum?']


9